In [2]:
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3, VGG16

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

import os
import requests
import zipfile
import glob

from sklearn.model_selection import train_test_split

from os import listdir
from os.path import isdir, isfile, join

In [3]:
target_directory = 'data/'

zip_url = 'https://www.muratkoklu.com/datasets/vtdhnd10.php'

os.makedirs(target_directory, exist_ok=True)
response = requests.get(zip_url)
zip_path = os.path.join(target_directory, 'data.zip')

with open(zip_path, 'wb') as zip_file:
    zip_file.write(response.content)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_directory)

In [4]:
data_dir = os.path.join(target_directory, 'Grapevine_Leaves_Image_Dataset/*')

data = []
labels = []


for class_folder in glob.glob(data_dir):
    label = os.path.basename(class_folder)

    for image_path in glob.glob(os.path.join(class_folder, '*.png')):
        labels.append(label)
        data.append(image_path)

In [5]:
print(len(data))

500


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)


In [7]:
print(X_train)



['data/Grapevine_Leaves_Image_Dataset/Ak/Ak (81).png', 'data/Grapevine_Leaves_Image_Dataset/Ala_Idris/Ala_Idris (68).png', 'data/Grapevine_Leaves_Image_Dataset/Buzgulu/Buzgulu (73).png', 'data/Grapevine_Leaves_Image_Dataset/Dimnit/Dimnit (43).png', 'data/Grapevine_Leaves_Image_Dataset/Ak/Ak (65).png', 'data/Grapevine_Leaves_Image_Dataset/Buzgulu/Buzgulu (47).png', 'data/Grapevine_Leaves_Image_Dataset/Nazli/Nazli (34).png', 'data/Grapevine_Leaves_Image_Dataset/Buzgulu/Buzgulu (39).png', 'data/Grapevine_Leaves_Image_Dataset/Dimnit/Dimnit (70).png', 'data/Grapevine_Leaves_Image_Dataset/Nazli/Nazli (29).png', 'data/Grapevine_Leaves_Image_Dataset/Ak/Ak (52).png', 'data/Grapevine_Leaves_Image_Dataset/Ala_Idris/Ala_Idris (89).png', 'data/Grapevine_Leaves_Image_Dataset/Buzgulu/Buzgulu (88).png', 'data/Grapevine_Leaves_Image_Dataset/Nazli/Nazli (95).png', 'data/Grapevine_Leaves_Image_Dataset/Nazli/Nazli (13).png', 'data/Grapevine_Leaves_Image_Dataset/Ala_Idris/Ala_Idris (58).png', 'data/Grapevi

In [8]:
img_width = 300
img_height = 300

EPOCHS = 20
BATCH_SIZE = 32

IMAGE_SHAPE = (img_width, img_height)

In [9]:
datagen = ImageDataGenerator(
    rescale=1/255.,
    validation_split=0.1
)

In [10]:
train_gen = datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_train, 'label': y_train}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    subset='training',
    class_mode='categorical',
    shuffle=True
)

validation_gen = datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_train, 'label': y_train}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SHAPE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

test_gen = datagen.flow_from_dataframe(
    pd.DataFrame({'image_path': X_test, 'label': y_test}),
    x_col='image_path',
    y_col='label',
    target_size=IMAGE_SHAPE,
    batch_size=1,
    class_mode=None,
    shuffle=False
)

Found 360 validated image filenames belonging to 5 classes.
Found 40 validated image filenames belonging to 5 classes.
Found 100 validated image filenames.


In [11]:
CLASSES = set(y_train)

# BASIC SEQUENTIAL MODEL

In [12]:
model = Sequential()

# Add a 2D convolutional layer with 32 filters of size 3x3 and ReLU activation function
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))

# Add a MaxPooling layer to reduce the data size
model.add(MaxPooling2D((2, 2)))

# Add another 2D convolutional layer with 64 filters of size 3x3 and ReLU activation
model.add(Conv2D(64, (3, 3), activation='relu'))

# Add another MaxPooling layer
model.add(MaxPooling2D((2, 2)))

# Add a 2D convolutional layer with 128 filters of size 3x3 and ReLU activation
model.add(Conv2D(128, (3, 3), activation='relu'))

# Add another MaxPooling layer
model.add(MaxPooling2D((2, 2)))

# Flatten the data into a one-dimensional array
model.add(Flatten())

# Add a dense layer with 512 units and ReLU activation
model.add(Dense(512, activation='relu'))

# Add an output layer with one neuron and a sigmoid activation function (binary classification)
model.add(Dense(len(CLASSES), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])


In [13]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=EPOCHS,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)
)

# STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
# STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

# history = model.fit_generator(generator=train_gen,
#                     steps_per_epoch=STEP_SIZE_TRAIN,
#                     validation_data=validation_gen,
#                     validation_steps=STEP_SIZE_VALID,
#                     epochs=EPOCHS
# )

Epoch 1/20
12/12 [==============================] - 17s 339ms/step - loss: 2.1375 - accuracy: 0.1861 - val_loss: 1.6941 - val_accuracy: 0.2000
Epoch 2/20
12/12 [==============================] - 3s 262ms/step - loss: 1.6256 - accuracy: 0.2389 - val_loss: 1.5866 - val_accuracy: 0.2500
Epoch 3/20
12/12 [==============================] - 3s 271ms/step - loss: 1.5855 - accuracy: 0.2500 - val_loss: 1.5833 - val_accuracy: 0.3250
Epoch 4/20
12/12 [==============================] - 3s 258ms/step - loss: 1.5436 - accuracy: 0.4083 - val_loss: 1.5597 - val_accuracy: 0.3000
Epoch 5/20
12/12 [==============================] - 3s 262ms/step - loss: 1.4631 - accuracy: 0.4778 - val_loss: 1.5231 - val_accuracy: 0.3000
Epoch 6/20
12/12 [==============================] - 3s 266ms/step - loss: 1.3656 - accuracy: 0.5083 - val_loss: 1.4629 - val_accuracy: 0.3500
Epoch 7/20
12/12 [==============================] - 3s 260ms/step - loss: 1.2482 - accuracy: 0.5583 - val_loss: 1.4756 - val_accuracy: 0.2250
Epoch

In [14]:
model.save('basic_model.keras')

# Model 1

### WITH DROPOUTS, BATCH_NORM AND EARLY STOPPING

In [15]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(CLASSES), activation='softmax'))

In [16]:
# Compile the model
opt = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

STEP_SIZE_TRAIN=train_gen.n//train_gen.batch_size
STEP_SIZE_VALID=validation_gen.n//validation_gen.batch_size

history = model.fit_generator(generator=train_gen,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_gen,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=EPOCHS,
                    callbacks=[callback],
                    verbose=1
)

<ipython-input-16-22eb5d98f54d>:10: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_gen,


Epoch 1/20
11/11 [==============================] - 6s 375ms/step - loss: 2.8974 - accuracy: 0.2683 - val_loss: 18.0500 - val_accuracy: 0.2812
Epoch 2/20
11/11 [==============================] - 4s 320ms/step - loss: 1.4269 - accuracy: 0.4817 - val_loss: 24.4584 - val_accuracy: 0.1875
Epoch 3/20
11/11 [==============================] - 4s 328ms/step - loss: 0.9470 - accuracy: 0.6311 - val_loss: 28.8623 - val_accuracy: 0.1562
Epoch 4/20
11/11 [==============================] - 4s 311ms/step - loss: 0.7473 - accuracy: 0.7195 - val_loss: 21.3096 - val_accuracy: 0.2188
Epoch 5/20
11/11 [==============================] - 4s 376ms/step - loss: 0.4697 - accuracy: 0.8323 - val_loss: 16.1652 - val_accuracy: 0.1250
Epoch 6/20
11/11 [==============================] - 3s 297ms/step - loss: 0.4075 - accuracy: 0.8598 - val_loss: 15.5539 - val_accuracy: 0.1562
Epoch 7/20
11/11 [==============================] - 4s 322ms/step - loss: 0.3153 - accuracy: 0.9085 - val_loss: 20.9922 - val_accuracy: 0.2500

In [17]:
model.save('model1.keras')

In [18]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(CLASSES), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Model 2

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_gen),
    epochs=EPOCHS,
    validation_data=validation_gen,
    validation_steps=len(validation_gen)
)

Epoch 1/20
13/13 [==============================] - 261s 19s/step - loss: 2.0993 - accuracy: 0.3704 - val_loss: 1.7191 - val_accuracy: 0.2444
Epoch 2/20
13/13 [==============================] - 228s 17s/step - loss: 0.6250 - accuracy: 0.7951 - val_loss: 2.3985 - val_accuracy: 0.2000
Epoch 3/20
13/13 [==============================] - 199s 15s/step - loss: 0.2955 - accuracy: 0.9210 - val_loss: 3.5069 - val_accuracy: 0.2000
Epoch 4/20
13/13 [==============================] - 190s 15s/step - loss: 0.1333 - accuracy: 0.9877 - val_loss: 4.5011 - val_accuracy: 0.2000
Epoch 5/20
13/13 [==============================] - 188s 14s/step - loss: 0.0619 - accuracy: 0.9926 - val_loss: 4.9750 - val_accuracy: 0.2000
Epoch 6/20
13/13 [==============================] - 204s 16s/step - loss: 0.0322 - accuracy: 0.9975 - val_loss: 4.8684 - val_accuracy: 0.2000
Epoch 7/20
13/13 [==============================] - 207s 15s/step - loss: 0.0198 - accuracy: 0.9975 - val_loss: 5.0436 - val_accuracy: 0.1556
Epoch 

KeyboardInterrupt: 